# Chapter 11 나이브 베이지안 분류기

## 02 베이즈 분류기 구현하기

'viagra' 단어가 들어가 있을 때 해당 메일이 스팸 메일일 확률 P(spam|viagra)

P(viagra) = 6/20

P(spam) = 6/20

P(viagra|spam) = 3/6

=> viagra라는 글자가 존재할 때 해당 메일이 스팸일 확률 P(spam|viagra) = 1/2

In [1]:
from pandas import Series, DataFrame
import pandas as pd
import numpy as np

viagra_spam={'viagra':[1,0,0,0,0,0,0,0,1,1,1,0,0,1,0,0,0,0,0,1],
            'spam':[1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,1,1,1]}
df=pd.DataFrame(viagra_spam,columns=['viagra','spam'])
np_data=df.values
np_data

array([[1, 1],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 1],
       [0, 0],
       [1, 1],
       [1, 0],
       [1, 0],
       [0, 0],
       [0, 0],
       [1, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 1],
       [0, 1],
       [1, 1]], dtype=int64)

viagra 단어를 포함하면서 스팸메일인 경우의 확률: P(viagra∩spam)

In [2]:
sum((np_data[:,0]==1) & (np_data[:,1]==1))/20

0.15

P(viagra)

In [3]:
p_viagra=sum(np_data[:,0]==1)/len(np_data)
p_viagra # P(viagra)

0.3

P(spam)

In [4]:
p_spam=sum(np_data[:,1]==1)/len(np_data)
p_spam # P(spam)

0.3

P(viagra∩spam)

In [5]:
p_v_cap_s=sum((np_data[:,0]==1) & (np_data[:,1]==1))/len(np_data)
p_v_cap_s # P(viagra∩spam)

0.15

P(~viagra∩spam)

In [6]:
p_n_v_cap_s=sum((np_data[:,0]==0) & (np_data[:,1]==1))/len(np_data)
p_n_v_cap_s # P(~viagra∩spam)

0.15

viagra 라는 단어가 들어갔을 때 스팸메일일 확률: P(spam|viagra)

In [7]:
p_spam * (p_v_cap_s/p_spam)/p_viagra

0.5

viagra 라는 단어가 들어가지 않았을 때 스팸메일일 확률: P(spam|~viagra)

In [8]:
p_spam * (p_v_cap_s/p_spam)/(1-p_viagra)

0.2142857142857143

viagra 라는 단어가 포함되었을 때 스팸메일일 확률(=0.5)은

viagra 라는 단어가 포함되지 않았을 때 스팸메일일 확률(=0.2142857142857143)보다 높다.

=> 이 결과로만 봤을 때, viagra라는 단어가 있으면 스팸메일로 분류하는 것이 좀 더 합리적인 선택이다.

## 03 나이브 베이지안 분류기 구현하기
#### 베이즈 분류기가 하나의 변수만을 고려하는 것에 반해 나이브 베이지안 분류기는 여러 개의 열을 사용하여 분류기를 구성한다.

In [9]:
from pandas import Series, DataFrame
import pandas as pd
import numpy as np

data_url="c:/Users/user/ch11/fraud.csv"
df=pd.read_csv(data_url,sep=',')
df.head()

,ID,History,CoApplicant,Accommodation,Fraud
0,1,current,none,own,True
1,2,paid,none,own,False
2,3,paid,none,own,False
3,4,paid,guarantor,rent,True
4,5,arrears,none,own,False


In [10]:
del df["ID"] # ID열 삭제
Y_data = df.pop("Fraud") # Fraud열 pop으로 원본에서 삭제시키고 Y_data에 Fraud열 저장
Y_data = Y_data.values
x_df = pd.get_dummies(df) # 원핫인코딩
x_df.head(10).T

,0,1,2,3,4,5,6,7,8,9
History_arrears,0,0,0,0,1,1,0,1,0,0
History_current,1,0,0,0,0,0,1,0,1,0
History_none,0,0,0,0,0,0,0,0,0,1
History_paid,0,1,1,1,0,0,0,0,0,0
CoApplicant_coapplicant,0,0,0,0,0,0,0,0,0,0
CoApplicant_guarantor,0,0,0,1,0,0,0,0,0,0
CoApplicant_none,1,1,1,0,1,1,1,1,1,1
Accommodation_free,0,0,0,0,0,0,0,0,0,0
Accommodation_own,1,1,1,0,1,1,1,1,0,1
Accommodation_rent,0,0,0,1,0,0,0,0,1,0


In [11]:
x_data = x_df.values # x_df의 값을 넘파이 배열 형태로 저장
x_data

array([[0, 1, 0, 0, 0, 0, 1, 0, 1, 0],
       [0, 0, 0, 1, 0, 0, 1, 0, 1, 0],
       [0, 0, 0, 1, 0, 0, 1, 0, 1, 0],
       [0, 0, 0, 1, 0, 1, 0, 0, 0, 1],
       [1, 0, 0, 0, 0, 0, 1, 0, 1, 0],
       [1, 0, 0, 0, 0, 0, 1, 0, 1, 0],
       [0, 1, 0, 0, 0, 0, 1, 0, 1, 0],
       [1, 0, 0, 0, 0, 0, 1, 0, 1, 0],
       [0, 1, 0, 0, 0, 0, 1, 0, 0, 1],
       [0, 0, 1, 0, 0, 0, 1, 0, 1, 0],
       [0, 1, 0, 0, 1, 0, 0, 0, 1, 0],
       [0, 1, 0, 0, 0, 0, 1, 0, 1, 0],
       [0, 1, 0, 0, 0, 0, 1, 0, 0, 1],
       [0, 0, 0, 1, 0, 0, 1, 0, 1, 0],
       [1, 0, 0, 0, 0, 0, 1, 0, 1, 0],
       [0, 1, 0, 0, 0, 0, 1, 0, 1, 0],
       [1, 0, 0, 0, 1, 0, 0, 0, 0, 1],
       [1, 0, 0, 0, 0, 0, 1, 1, 0, 0],
       [1, 0, 0, 0, 0, 0, 1, 0, 1, 0],
       [0, 0, 0, 1, 0, 0, 1, 0, 1, 0]], dtype=uint8)

나이브 베이지안 수식을 코드로 표현하기

In [46]:
P_Y_True = sum(Y_data==True) / len(Y_data)
P_Y_False = 1 - P_Y_True

P_Y_True,P_Y_False # Y값이 True인 경우와 False인 경우의 확률

(0.3, 0.7)

- np.where 함수: 특정 조건을 만족하는 값의 인덱스를 반환하는 함수

In [13]:
np.where(Y_data)

(array([ 0,  3,  5,  9, 11, 12], dtype=int64),)

y_data=True,False인 각각의 index

In [14]:
ix_Y_True = np.where(Y_data)
ix_Y_False = np.where(Y_data==False)

ix_Y_True, ix_Y_False

((array([ 0,  3,  5,  9, 11, 12], dtype=int64),),
 (array([ 1,  2,  4,  6,  7,  8, 10, 13, 14, 15, 16, 17, 18, 19],
        dtype=int64),))

In [15]:
p_x_y_true = (x_data[ix_Y_True].sum(axis=0)) / sum(Y_data==True) # y_data=True인 인덱스에 존재하는 x_data의 합을 y_data=True인 개수(6개)로 나눠주기
p_x_y_false = (x_data[ix_Y_False].sum(axis=0)) / sum(Y_data==False) # y_data=False인 인덱스에 존재하는 x_data의 합을 y_data=False인 개수(14개)로 나눠주기

p_x_y_true, p_x_y_false

(array([0.16666667, 0.5       , 0.16666667, 0.16666667, 0.        ,
        0.16666667, 0.83333333, 0.        , 0.66666667, 0.33333333]),
 array([0.42857143, 0.28571429, 0.        , 0.28571429, 0.14285714,
        0.        , 0.85714286, 0.07142857, 0.78571429, 0.14285714]))

In [16]:
x_test = [0,1,0,0,0,1,0, 0,1,0]

p_y_true_test = P_Y_True + p_x_y_true.dot(x_test)
p_y_false_test = P_Y_False + p_x_y_false.dot(x_test)

p_y_true_test , p_y_false_test

(1.6333333333333333, 1.7714285714285714)

In [17]:
p_y_true_test < p_y_false_test # false_test가 더 크다.

True

#### 하나의 문장이 있을 때 이 문장을 sports와 not sports로 나누는 분류기 만들기

In [18]:
y_example_text = ["Sports", "Not sports","Sports","Sports","Not sports"]
y_example = [1 if c=="Sports" else 0 for c in y_example_text ] # sports가 y_example_text에 있으면 1 없으면 0
print(y_example)
text_example = ["A great game game", "The The election was over",
                "Very clean game match",
                "A clean but forgettable game game",
                "It was a close election", ]

[1, 0, 1, 1, 0]


- BoW(Bag of Words): 단어별로 인덱스가 부여되어 있을 때 한 문장 또는 한 문서에 대한 벡터를 표현하는 기법

In [19]:
from sklearn.feature_extraction.text import CountVectorizer

countvect_example = CountVectorizer() # BoW 벡터를 생성하기 위한 클래스
X_example = countvect_example.fit_transform(text_example)
countvect_example.get_feature_names() # 처리해야 하는 단어들의 모음을 보여줌 = text_example 객체에 존재하는 단어들의 모음

C:\Users\user\anaconda3\envs\ml\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


['but',
 'clean',
 'close',
 'election',
 'forgettable',
 'game',
 'great',
 'it',
 'match',
 'over',
 'the',
 'very',
 'was']

In [20]:
countvect_example.transform(text_example).toarray() # 5개 문장에 대한 각 단어들의 출현한 개수

array([[0, 0, 0, 0, 0, 2, 1, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 2, 0, 1],
       [0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0],
       [1, 1, 0, 0, 1, 2, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1]], dtype=int64)

In [21]:
countvect_example.vocabulary_ # 각 단어들의 인덱스 번호 확인

{'great': 6,
 'game': 5,
 'the': 10,
 'election': 3,
 'was': 12,
 'over': 9,
 'very': 11,
 'clean': 1,
 'match': 8,
 'but': 0,
 'forgettable': 4,
 'it': 7,
 'close': 2}

#### 베르누이 나이브 베이지안 분류기
- 다루고자 하는 모든 데이터가 불린 피쳐
- 사용되는 데이터 타입은 이산형 데이터인데, 이러한 데이터를 모두 불린 타입으로 변경하여 학습
    - 정수 타입 숫자라면 임계값 기준으로 True 또는 Fasle로 변환

In [22]:
from sklearn.naive_bayes import BernoulliNB

clf = BernoulliNB(alpha=1, binarize=0) # 모델 만들기
clf.fit(X_example, y_example)

BernoulliNB(alpha=1, binarize=0)

In [23]:
clf.class_log_prior_ # 각 클래스마다 prior의 값에 log를 붙여서 값을 출력

array([-0.91629073, -0.51082562])

#### 다항 나이브 베이지안 분류기
- 베르누이 분류기와 달리 각 피쳐들이 이산형이지만, 이진값이 아닌 여러 개의 값을 가질 수 있다.

In [24]:
from sklearn.naive_bayes import MultinomialNB

clf = MultinomialNB(alpha=1)
clf.fit(X_example, y_example)

MultinomialNB(alpha=1)

#### 가우시안 나이브 베이지안 분류기

In [25]:
from sklearn.naive_bayes import GaussianNB

clf = GaussianNB()
clf.fit(X_example.toarray(), y_example)

GaussianNB()

## 04 20newsgroup으로 분류 연습하기
#### 20newsgroup 데이터셋을 사용하여 나이브 베이지안 분류기로 20개의 뉴스 텍스트 데이터 분류하기

In [26]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
%matplotlib inline

##### 데이터셋 불러오기

In [28]:
from sklearn.datasets import fetch_20newsgroups
news=fetch_20newsgroups(subset='all')
news.keys()

dict_keys(['data', 'filenames', 'target_names', 'target', 'DESCR'])

- data:실제 데이터
- filenames: 다운로드된 데이터의 파일 위치
- target_names: 데이터 y값의 이름
- target: 데이터 y 값의 인덱스
- DESCR: 데이터에 대한 설명

In [29]:
print(news.data[0])

From: Mamatha Devineni Ratnam <mr47+@andrew.cmu.edu>
Subject: Pens fans reactions
Organization: Post Office, Carnegie Mellon, Pittsburgh, PA
Lines: 12
NNTP-Posting-Host: po4.andrew.cmu.edu



I am sure some bashers of Pens fans are pretty confused about the lack
of any kind of posts about the recent Pens massacre of the Devils. Actually,
I am  bit puzzled too and a bit relieved. However, I am going to put an end
to non-PIttsburghers' relief with a bit of praise for the Pens. Man, they
are killing those Devils worse than I thought. Jagr just showed you why
he is much better than his regular season stats. He is also a lot
fo fun to watch in the playoffs. Bowman should let JAgr have a lot of
fun in the next couple of games since the Pens are going to beat the pulp out of Jersey anyway. I was very disappointed not to see the Islanders lose the final
regular season game.          PENS RULE!!!




y값에 해당하는 target과 target_names 출력

In [32]:
news.target # 데이터 y값

array([10,  3, 17, ...,  3,  1,  7])

In [33]:
news.target_names # 데이터 y값의 인덱스

['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

##### 데이터 전처리

데이터 프레임 만들기

In [34]:
news_df=pd.DataFrame({'News':news.data,'Target':news.target}) # 데이터프레임 만들기
news_df.head()

,News,Target
0,From: Mamatha Devineni Ratnam <mr47+@andrew.cm...,10
1,From: mblawson@midway.ecn.uoknor.edu (Matthew ...,3
2,From: hilmi-er@dsv.su.se (Hilmi Eren)\nSubject...,17
3,From: guyd@austin.ibm.com (Guy Dawson)\nSubjec...,3
4,From: Alexander Samuel McDiarmid <am2o+@andrew...,4


Target의 정보를 실제 분류의 이름으로 변경하기

In [35]:
target_dict={idx:name for idx, name in enumerate(news.target_names)}
news_df['Target']=news_df['Target'].replace(target_dict) # Target열 값을 target_dict로 바꾸기

News열 전처리

In [36]:
def data_cleansing(df):
    delete_email = re.sub(r'\b[\w\+]+@[\w]+.[\w]+.[\w]+.[\w]+\b', ' ', df) # 이메일 제거
    delete_number = re.sub(r'\b|\d+|\b', ' ',delete_email) # 불필요 숫자 제거
    delete_non_word = re.sub(r'\b[\W]+\b', ' ', delete_number) # 문자 아닌 특수문자 제거
    cleaning_result = ' '.join(delete_non_word.split()) # 단어 사이 공백 제거:띄어쓰기별로 분할하고 결합
    return cleaning_result

news_df.loc[:,'News']=news_df['News'].apply(data_cleansing) # News열에 전처리된 텍스트 적용
news_df.head()

,News,Target
0,From Mamatha Devineni Ratnam Subject Pens fans...,rec.sport.hockey
1,From Matthew B Lawson Subject Which high perfo...,comp.sys.ibm.pc.hardware
2,From hilmi Hilmi Eren Subject Re ARMENIA SAYS ...,talk.politics.mideast
3,From Guy Dawson Subject Re IDE vs SCSI DMA and...,comp.sys.ibm.pc.hardware
4,From Alexander Samuel McDiarmid Subject driver...,comp.sys.mac.hardware


##### 벡터화 하기
토큰(token): 인덱스를 지정해야 하는 단어들의 리스트를 정리하는 기법
- 어간추출(stemming): 띄어쓰기 기준이 아닌 의미나 역할이 다른 단어들을 기준으로 분리하는 기법

In [37]:
!pip install nltk

In [38]:
from nltk import stem
stmmer=stem.SnowballStemmer('english')
sentence='looking looks looked'
[stmmer.stem(word) for word in sentence.split()]

['look', 'look', 'look']

In [40]:
stmmer.stem('images'),stmmer.stem('imaging'),stmmer.stem('imagination')

('imag', 'imag', 'imagin')

In [42]:
from sklearn.feature_extraction.text import CountVectorizer
import nltk

enlish_stemmer = nltk.stem.SnowballStemmer("english")
class StemmedCountVectorizer(CountVectorizer): # class 1
    def build_analyzer(self): # 함수 재정의
        analyzer = super(StemmedCountVectorizer,self).build_analyzer()
        return lambda doc: (enlish_stemmer.stem(w) for w in analyzer(doc))
    
from sklearn.feature_extraction.text import TfidfVectorizer

enlish_stemmer = nltk.stem.SnowballStemmer("english")
class StemmedTfidfVectorizer(TfidfVectorizer): # class 2
    def build_analyzer(self):
        analyzer = super(StemmedTfidfVectorizer,self).build_analyzer()
        return lambda doc: (enlish_stemmer.stem(w) for w in analyzer(doc))

##### 모델링하기


In [43]:
from sklearn.naive_bayes import MultinomialNB, BernoulliNB,GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.pipeline import make_pipeline

vectorizer = [CountVectorizer(), TfidfVectorizer(), StemmedCountVectorizer(), StemmedTfidfVectorizer()]
# algorithms = [BernoulliNB(), MultinomialNB(), GaussianNB(), LogisticRegression()]
algorithms = [MultinomialNB(), LogisticRegression()]

pipelines = []

In [44]:
import itertools
for case in list(itertools.product(vectorizer,algorithms)):
    pipelines.append(make_pipeline(*case))

In [45]:
ngrams_params = [(1,1),(1,3)]
stopword_params = ["english"]
lowercase_params = [True, False]
max_df_params = np.linspace(0.4, 0.6, num=6) # 0.4에서 0.6까지 6개로 나눔
min_df_params = np.linspace(0.0, 0.0, num=1)

attributes = {"ngram_range":ngrams_params, "max_df":max_df_params,"min_df":min_df_params,
              "lowercase":lowercase_params,"stop_words":stopword_params}
vectorizer_names = ["countvectorizer","tfidfvectorizer","stemmedcountvectorizer","stemmedtfidfvectorizer"]
vectorizer_params_dict = {}

for vect_name in vectorizer_names:
    vectorizer_params_dict[vect_name] = {}
    for key, value in attributes.items():
        param_name = vect_name + "__" + key
        vectorizer_params_dict[vect_name][param_name] = value